## 번역 데이터 준비

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

import re
import os
import random
import math

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

print(tf.__version__)

2.6.0


In [2]:
import numpy 
import pandas 
import tensorflow 
import nltk
import gensim

print(numpy.__version__)
print(pandas.__version__)
print(tensorflow.__version__)
print(nltk.__version__)
print(gensim.__version__)

1.21.4
1.3.3
2.6.0
3.6.5
4.1.2


챗봇 데이터 다운

In [3]:
zip_path = tf.keras.utils.get_file(
    'ChatbotData.csv',
    origin='https://github.com/songys/Chatbot_data.git',
    extract=True
)

print('슝=3')

 212992/Unknown - 0s 1us/step슝=3


중복치 제거

In [5]:
print(zip_path)  # zip_path가 정확한지 확인


/aiffel/.keras/datasets/ChatbotData.csv


In [8]:
file_path = "./data/ChatbotData.csv"

with open(file_path, "r") as f:
    spa_eng_sentences = f.read().splitlines()

spa_eng_sentences = list(set(spa_eng_sentences)) 
total_sentence_count = len(spa_eng_sentences)
print("Example:", total_sentence_count)

for sen in spa_eng_sentences[0:100][::20]: 
    print(">>", sen)


Example: 11824
>> 다 맞춰주고 싶은 사람,정말 사랑하고 있나봐요.,2
>> 나만 친구로 생각했나봐,그런 친구는 거르세요.,0
>> 짝녀가 나를 떠보는 걸까.,입장을 확실히 하는 편이 좋겠네요.,2
>> 기 빨렸어,너무 긴장했나봐요.,0
>> 마음도 춥고 날씨도 춥네,마음 감기조심하세요.,0


In [15]:
import pandas as pd

# CSV 파일 경로
file_path = "./data/ChatbotData.csv"

# pandas로 CSV 파일 읽기
data = pd.read_csv(file_path, header=None)

# 질문과 답변을 나눠서 저장
questions = data[0]  # 첫 번째 열이 질문
answers = data[1]    # 두 번째 열이 답변

# 전체 데이터 개수
total_sentence_count = len(questions)

# 테스트 데이터 개수 (전체 데이터의 1/200)
test_sentence_count = total_sentence_count // 200
print("Test Size: ", test_sentence_count)
print("\n")

# Train/Test 데이터 나누기
train_questions = questions[:-test_sentence_count]
test_questions = questions[-test_sentence_count:]
train_answers = answers[:-test_sentence_count]
test_answers = answers[-test_sentence_count:]

# Train 데이터 확인
print("Train Questions Example:", len(train_questions))
for sen in train_questions[0:100][::20]:
    print(">>", sen)
print("\nTrain Answers Example:", len(train_answers))
for sen in train_answers[0:100][::20]:
    print(">>", sen)
print("\n")

# Test 데이터 확인
print("Test Questions Example:", len(test_questions))
for sen in test_questions[0:100][::20]:
    print(">>", sen)
print("\nTest Answers Example:", len(test_answers))
for sen in test_answers[0:100][::20]:
    print(">>", sen)


Test Size:  59


Train Questions Example: 11765
>> Q
>> 가스비 너무 많이 나왔다.
>> 가출해도 갈 데가 없어
>> 감정이 쓰레기통처럼 엉망진창이야
>> 같이 할 수 있는 취미 생활 뭐 있을까

Train Answers Example: 11765
>> A
>> 다음 달에는 더 절약해봐요.
>> 선생님이나 기관에 연락해보세요.
>> 자신을 더 사랑해주세요.
>> 함께하면 서로를 더 많이 알게 될 거예요.


Test Questions Example: 59
>> 학교샘 좋아하는 사람 있나?
>> 헤어지고 나서 알았어 사랑했다는 걸
>> 혼자가 편하다는 짝남에게 먼저 대쉬해버림.

Test Answers Example: 59
>> 존경의 의미라고 생각하세요.
>> 진정한 사랑이라고 생각한다면 다시 연락해보세요.
>> 거절의 뜻은 아니었나요.


전처리

In [11]:
# Q. 전처리 함수를 만들어 보세요. 아래 기능을 추가해주세요.
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower()  # 모든 문자를 소문자로 변환
    sentence = re.sub(r'[^a-zA-Z0-9가-힣.,!? ]+', '', sentence)  # 영문자, 숫자, 한글, 주요 특수문자를 제외하고 제거
    sentence = re.sub(r'\s+', ' ', sentence)  # 둘 이상의 공백을 하나로 치환
    sentence = sentence.strip()  # 문자열 양 끝 공백 제거
    return sentence

# 테스트
test_sentence = "안녕하세요! This is a TEST 문장입니다. Numbers: 1234, 특수기호: #@$%^&*()"
processed_sentence = preprocess_sentence(test_sentence)
print(processed_sentence)



안녕하세요! this is a test 문장입니다. numbers 1234, 특수기호


In [12]:
spa_eng_sentences = list(map(preprocess_sentence, spa_eng_sentences))

print('슝=3')

슝=3


In [16]:
# 전체 질문과 답변 개수 계산
total_sentence_count = len(questions)
test_sentence_count = total_sentence_count // 200
print("Test Size: ", test_sentence_count)
print("\n")

# Train/Test 데이터 나누기 - 질문
train_questions = questions[:-test_sentence_count]
test_questions = questions[-test_sentence_count:]

# Train/Test 데이터 나누기 - 답변
train_answers = answers[:-test_sentence_count]
test_answers = answers[-test_sentence_count:]

# Train 데이터 확인 (질문과 답변)
print("Train Questions Example:", len(train_questions))
for sen in train_questions[0:100][::20]: 
    print(">>", sen)
print("\n")

print("Train Answers Example:", len(train_answers))
for sen in train_answers[0:100][::20]: 
    print(">>", sen)
print("\n")

# Test 데이터 확인 (질문과 답변)
print("Test Questions Example:", len(test_questions))
for sen in test_questions[0:100][::20]: 
    print(">>", sen)
print("\n")

print("Test Answers Example:", len(test_answers))
for sen in test_answers[0:100][::20]: 
    print(">>", sen)
print("\n")


Test Size:  59


Train Questions Example: 11765
>> Q
>> 가스비 너무 많이 나왔다.
>> 가출해도 갈 데가 없어
>> 감정이 쓰레기통처럼 엉망진창이야
>> 같이 할 수 있는 취미 생활 뭐 있을까


Train Answers Example: 11765
>> A
>> 다음 달에는 더 절약해봐요.
>> 선생님이나 기관에 연락해보세요.
>> 자신을 더 사랑해주세요.
>> 함께하면 서로를 더 많이 알게 될 거예요.


Test Questions Example: 59
>> 학교샘 좋아하는 사람 있나?
>> 헤어지고 나서 알았어 사랑했다는 걸
>> 혼자가 편하다는 짝남에게 먼저 대쉬해버림.


Test Answers Example: 59
>> 존경의 의미라고 생각하세요.
>> 진정한 사랑이라고 생각한다면 다시 연락해보세요.
>> 거절의 뜻은 아니었나요.




In [17]:
def split_spa_eng_sentences(spa_eng_sentences):
    spa_sentences = []
    eng_sentences = []
    for spa_eng_sentence in tqdm(spa_eng_sentences):
        eng_sentence, spa_sentence = spa_eng_sentence.split('\t')
        spa_sentences.append(spa_sentence)
        eng_sentences.append(eng_sentence)
    return eng_sentences, spa_sentences

print('슝=3')

슝=3


토큰화

In [20]:
def generate_tokenizer(corpus,
                       vocab_size,
                       lang="spa-eng",
                       pad_id=0,   # pad token의 일련번호
                       bos_id=1,  # 문장의 시작을 의미하는 bos token(<s>)의 일련번호
                       eos_id=2,  # 문장의 끝을 의미하는 eos token(</s>)의 일련번호
                       unk_id=3):   # unk token의 일련번호
    file = "./%s_corpus.txt" % lang
    model = "%s_spm" % lang

    with open(file, 'w') as f:
        for row in corpus: f.write(str(row) + '\n')

    import sentencepiece as spm
    spm.SentencePieceTrainer.Train(
        '--input=./%s --model_prefix=%s --vocab_size=%d'\
        % (file, model, vocab_size) + \
        '--pad_id==%d --bos_id=%d --eos_id=%d --unk_id=%d'\
        % (pad_id, bos_id, eos_id, unk_id)
    )

    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load('%s.model' % model)

    return tokenizer

print("슝=3")

슝=3


In [ ]:
#!pip install transformers torch


## 동의어 치환 증강

In [119]:
import random

# 동의어 딕셔너리 예시
synonym_dict = {
    "좋다": ["훌륭하다", "멋지다", "아름답다", "재미있다", "신나다"],
    "나쁘다": ["형편없다", "끔찍하다", "불쾌하다"],
    "어렵다": ["힘들다", "어렵다", "복잡하다"],
    "쉽다": ["간단하다", "수월하다"],
    "사랑하다": ["좋아하다", "애정하다", "선호하다"],
    # 추가 단어와 동의어 정의
}

def synonym_replacement(sentence):
    words = sentence.split()
    new_words = words.copy()
    
    for i, word in enumerate(words):
        if word in synonym_dict:
            synonym = random.choice(synonym_dict[word])
            new_words[i] = synonym

    return ' '.join(new_words)

# 예시 문장 증강
augmented_questions = [synonym_replacement(q) for q in train_questions]
augmented_answers = [synonym_replacement(a) for a in train_answers]


In [121]:
# 원래 데이터와 증강 데이터를 합침
all_questions = train_questions + augmented_questions
all_answers = train_answers + augmented_answers

# vocab_size를 적절히 조정 (예: 10000)
VOCAB_SIZE = min(30000, len(set(all_questions + all_answers)))

# 새로운 토크나이저 생성
tokenizer = generate_tokenizer(all_questions + all_answers, VOCAB_SIZE, 'questions-answers')
tokenizer.set_encode_extra_options("bos:eos")  # 문장 양 끝에 <s>, </s> 추가


sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=././questions-answers_corpus.txt --model_prefix=questions-answers_spm --vocab_size=11692--pad_id==0 --bos_id=1 --eos_id=2 --unk_id=3
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ././questions-answers_corpus.txt
  input_format: 
  model_prefix: questions-answers_spm
  model_type: UNIGRAM
  vocab_size: 11692
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id

True

move_extra_whitespaces: 1
  escape_whitespaces: 1
  normalization_rule_tsv: 
}
denormalizer_spec {}
trainer_interface.cc(329) LOG(INFO) SentenceIterator is not specified. Using MultiFileSentenceIterator.
trainer_interface.cc(178) LOG(INFO) Loading corpus: ././questions-answers_corpus.txt
trainer_interface.cc(385) LOG(INFO) Loaded all 11765 sentences
trainer_interface.cc(400) LOG(INFO) Adding meta_piece: <s>
trainer_interface.cc(400) LOG(INFO) Adding meta_piece: </s>
trainer_interface.cc(400) LOG(INFO) Adding meta_piece: <unk>
trainer_interface.cc(405) LOG(INFO) Normalizing sentences...
trainer_interface.cc(466) LOG(INFO) all chars count=666984
trainer_interface.cc(477) LOG(INFO) Done: 99.9505% characters are covered.
trainer_interface.cc(487) LOG(INFO) Alphabet size=1090
trainer_interface.cc(488) LOG(INFO) Final character coverage=0.999505
trainer_interface.cc(520) LOG(INFO) Done! preprocessed 11765 sentences.
unigram_model_trainer.cc(139) LOG(INFO) Making suffix array...
unigram_model

In [126]:
VOCAB_SIZE = 10000  # 가능한 단어 수에 맞춰 설정
tokenizer = generate_tokenizer(train_questions + train_answers, VOCAB_SIZE, 'questions-answers')
tokenizer.set_encode_extra_options("bos:eos")  # 문장 양 끝에 <s>, </s> 추가


sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=././questions-answers_corpus.txt --model_prefix=questions-answers_spm --vocab_size=10000--pad_id==0 --bos_id=1 --eos_id=2 --unk_id=3
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ././questions-answers_corpus.txt
  input_format: 
  model_prefix: questions-answers_spm
  model_type: UNIGRAM
  vocab_size: 10000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id

True

ge=0.999502
trainer_interface.cc(520) LOG(INFO) Done! preprocessed 11765 sentences.
unigram_model_trainer.cc(139) LOG(INFO) Making suffix array...
unigram_model_trainer.cc(143) LOG(INFO) Extracting frequent sub strings...
unigram_model_trainer.cc(194) LOG(INFO) Initialized 21294 seed sentencepieces
trainer_interface.cc(526) LOG(INFO) Tokenizing input sentences with whitespace: 11765
trainer_interface.cc(537) LOG(INFO) Done! 25859
unigram_model_trainer.cc(489) LOG(INFO) Using 25859 sentences for EM training
unigram_model_trainer.cc(505) LOG(INFO) EM sub_iter=0 size=12509 obj=15.3532 num_tokens=66941 num_tokens/piece=5.35143
unigram_model_trainer.cc(505) LOG(INFO) EM sub_iter=1 size=11501 obj=14.2166 num_tokens=67163 num_tokens/piece=5.83975
unigram_model_trainer.cc(505) LOG(INFO) EM sub_iter=0 size=10995 obj=14.2136 num_tokens=67416 num_tokens/piece=6.13151
unigram_model_trainer.cc(505) LOG(INFO) EM sub_iter=1 size=10944 obj=14.1978 num_tokens=67480 num_tokens/piece=6.16594
trainer_inte

In [23]:
def make_corpus(sentences, tokenizer):
    corpus = []
    for sentence in tqdm(sentences):
        tokens = tokenizer.encode_as_ids(sentence)
        corpus.append(tokens)
    return corpus

print('슝=3')

슝=3


In [25]:
questions_corpus = make_corpus(train_questions, tokenizer)  # 질문 데이터로 코퍼스 생성
answers_corpus = make_corpus(train_answers, tokenizer)      # 답변 데이터로 코퍼스 생성


  0%|          | 0/11765 [00:00<?, ?it/s]

  0%|          | 0/11765 [00:00<?, ?it/s]

In [26]:
print(train_questions[0])  # 첫 번째 질문 출력
print(questions_corpus[0])  # 첫 번째 질문 코퍼스 출력
print('\n')
print(train_answers[0])  # 첫 번째 답변 출력
print(answers_corpus[0])  # 첫 번째 답변 코퍼스 출력


Q
[1, 4, 3, 2]


A
[1, 4901, 2]


In [27]:
MAX_LEN = 50
questions_ndarray = tf.keras.preprocessing.sequence.pad_sequences(questions_corpus, maxlen=MAX_LEN, padding='post')
answers_ndarray = tf.keras.preprocessing.sequence.pad_sequences(answers_corpus, maxlen=MAX_LEN, padding='post')

print('슝=3')


슝=3


In [28]:
BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices((questions_ndarray, answers_ndarray)).batch(batch_size=BATCH_SIZE)

print('슝=3')

슝=3


## 번역 모델 만들기

positional Encoding

In [79]:
# Positional Encoding 구현
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

    return sinusoid_table
print("슝=3")

슝=3


In [80]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [81]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights

In [82]:

class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out

In [83]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

In [84]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        '''
        Masked Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        '''
        Multi-Head Attention
        '''
        residual = out
        out = self.norm_2(out)
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [85]:
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

In [86]:
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [87]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

In [88]:
transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)
		
d_model = 512

In [89]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [99]:
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [100]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [103]:
def train_step(questions_batch, answers_batch, optimizer):
    enc_padding_mask, dec_padding_mask, look_ahead_mask = create_masks(questions_batch, answers_batch)
    
    with tf.GradientTape() as tape:
        predictions = transformer(questions_batch, answers_batch, 
                                  enc_padding_mask, 
                                  dec_padding_mask, 
                                  look_ahead_mask, 
                                  training=True)

        # predictions가 tuple인 경우 첫 번째 요소를 선택
        if isinstance(predictions, tuple):
            predictions = predictions[0]  # 필요한 요소로 변경
        
        loss = loss_fn(answers_batch, predictions)

    # 가중치 업데이트
    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    return loss


In [104]:
EPOCHS = 3

for epoch in range(EPOCHS):
    total_loss = 0

    dataset_count = tf.data.experimental.cardinality(train_dataset).numpy()
    tqdm_bar = tqdm(total=dataset_count)

    # train_dataset에서 배치를 가져오도록 수정
    for questions_batch, answers_batch in train_dataset:
        batch_loss = train_step(questions_batch, answers_batch, optimizer)
        total_loss += batch_loss.numpy()

        
        # tqdm 진행 표시줄 업데이트
        tqdm_bar.update(1)  # 한 배치 처리 후 업데이트
        tqdm_bar.set_description_str('Epoch %2d' % (epoch + 1))
        tqdm_bar.set_postfix_str('Loss %.4f' % (total_loss / (tqdm_bar.n + 1)))

    tqdm_bar.close()  # tqdm 종료

# 모델 훈련 후 결과 출력
print(f"Training completed for {EPOCHS} epochs. Final loss: {total_loss / dataset_count:.4f}")


  0%|          | 0/184 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

Training completed for 3 epochs. Final loss: 0.5548


In [ ]:
import tensorflow as tf
from tqdm import tqdm

# 모델 훈련 코드 (기존 코드)
EPOCHS = 3

for epoch in range(EPOCHS):
    total_loss = 0

    dataset_count = tf.data.experimental.cardinality(train_dataset).numpy()
    tqdm_bar = tqdm(total=dataset_count)

    for questions_batch, answers_batch in train_dataset:
        batch_loss = train_step(questions_batch, answers_batch, optimizer)
        total_loss += batch_loss.numpy()
        
        # tqdm 진행 표시줄 업데이트
        tqdm_bar.update(1)
        tqdm_bar.set_description_str('Epoch %2d' % (epoch + 1))
        tqdm_bar.set_postfix_str('Loss %.4f' % (total_loss / (tqdm_bar.n + 1)))

    tqdm_bar.close()

# 모델 훈련 후 결과 출력
print(f"Training completed for {EPOCHS} epochs. Final loss: {total_loss / dataset_count:.4f}")
